Взять датасет из google диска: https://drive.google.com/file/d/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_. Проверить гипотезу о том, в каком варианте теста (control/personalization) больше конверсия (converted) и значимо ли это отличие статистически.

In [1]:
# !wget 'https://drive.google.com/uc?export=download&id=1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_' -O data.zip

In [2]:
# !unzip data.zip

In [3]:
import numpy as np
import pandas as pd
from statsmodels.stats import proportion as pr

Качаем и посмотрим, что скачали:

In [4]:
df_marketing = pd.read_csv('marketing_campaign.csv', header = 0, sep = ',')
df_subscribers = pd.read_csv('subscribers.csv', header = 0, sep = ',')
df_users = pd.read_csv('users.csv', header = 0, sep = ',')

In [5]:
display('df_marketing:',  df_marketing.head(), 
        'df_subscribers:', df_subscribers.head(), 
        'df_users:', df_users.head())

'df_marketing:'

,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,1/1/18,House Ads,personalization,English,True
1,a1001,1/1/18,House Ads,personalization,English,True
2,a1002,1/1/18,House Ads,personalization,English,True
3,a1003,1/1/18,House Ads,personalization,English,True
4,a1004,1/1/18,House Ads,personalization,English,True


'df_subscribers:'

,user_id,subscribing_channel,date_subscribed,date_canceled,is_retained
0,a1000,House Ads,1/1/18,NaN,True
1,a1001,House Ads,1/1/18,NaN,True
2,a1002,House Ads,1/1/18,NaN,True
3,a1003,House Ads,1/1/18,NaN,True
4,a1004,House Ads,1/1/18,NaN,True


'df_users:'

,user_id,age_group,language_preferred
0,a1000,0-18 years,English
1,a1001,19-24 years,English
2,a1002,24-30 years,English
3,a1003,30-36 years,English
4,a1004,36-45 years,English


соберём всё, что есть, в единый датафрейм

In [6]:
df = df_marketing.merge(df_subscribers, on='user_id', how='inner').merge(df_users, on='user_id', how='inner')
df.head()

,user_id,date_served,marketing_channel,variant,language_displayed,converted,subscribing_channel,date_subscribed,date_canceled,is_retained,age_group,language_preferred
0,a1000,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True,0-18 years,English
1,a1001,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True,19-24 years,English
2,a1002,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True,24-30 years,English
3,a1003,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True,30-36 years,English
4,a1004,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True,36-45 years,English


Какие признаки есть:

- **```user_id```**: идентификатор пользователя
- **```date_served```**: дата события
- **```marketing_channel```**: рекламный канал
- **```variant```**: показанный вариант рекламы
- **```converted```**: конверсия в покупку, 1 - пользователь совершил покупку, 0 - нет
- **```language_displayed```**: язык рекламного сообщения
- **```language_preferred```**: предпочитаемый пользователем язык
- **```age_group```**: возрастная группа пользователя
- **```date_subscribed```**: дата подписки на сервис
- **```date_canceled```**: дата отказа от подписки
- **```subscribing_channel```**: с какого рекламного канал пришел пользователь, когда подписался на сервис
- **```is_retained```**: удержание, 1 - пользователь продолжает пользоваться услугами сервиса, 0 - пользователь отвалился 

In [7]:
df.isna().sum(), df.shape

(user_id                   0
 date_served              16
 marketing_channel        15
 variant                   0
 language_displayed        0
 converted                15
 subscribing_channel    8181
 date_subscribed        8181
 date_canceled          9460
 is_retained            8181
 age_group                 0
 language_preferred        0
 dtype: int64,
 (10037, 12))

файл ```subscribers.csv``` оказался наполовину дырявым, а пропуски в ```date_served```, ```marketing_channel``` и ```converted``` незначительные и в одних и тех же строках, удалим:

In [8]:
df.query('date_served.isna() | marketing_channel.isna() | converted.isna()').shape

(16, 12)

In [9]:
df = df.query('date_served.notnull() & marketing_channel.notnull() & converted.notnull()')
df.shape

(10021, 12)

считаем значения:

k1 и k2 - это количество "успехов" в двух группах данных. 

n1 и n2 - это общее количество наблюдений в каждой из двух групп. 

In [10]:
k1 = df.loc[df['variant']=='control', 'converted'].sum()
n1 = df[df['variant']=='control'].shape[0]
k2 = df.loc[df['variant']=='personalization', 'converted'].sum()
n2 = df[df['variant']=='personalization'].shape[0]
print(f'{k1=}\n{n1=}\n{k2=}\n{n2=}')

k1=371
n1=5076
k2=705
n2=4945


Формулируем гипотезы:

$H_0$: конвесии в ```control``` и ```personalization``` равны 

$H_1$: конвесии в ```control``` и ```personalization``` не равны

проведём Z-тест:

In [11]:
z_score, z_pvalue = pr.proportions_ztest(
    np.array([k1, k2]),
    np.array([n1, n2])
)
print(f'результат:\n{z_score=:.3f}\n{z_pvalue=:.3f}')
print('=> отвергаем H0') if z_pvalue < 0.05 else print('=> принимаем H0')

результат:
z_score=-11.232
z_pvalue=0.000
=> отвергаем H0


проведём Хи-квадрат-тест:

In [12]:
chisq, pvalue, table = pr.proportions_chisquare(
    np.array([k1, k2]),
    np.array([n1, n2])
)
print(f'результат:\n{chisq=:.3f}\n{pvalue=:.3f}')
print('=> отвергаем H0') if pvalue < 0.05 else print('=> принимаем H0')

результат:
chisq=126.158
pvalue=0.000
=> отвергаем H0


**Вывод**: по результатам стат.тестов мы не можем принять нулевую гипотезу $H_0$ 
, с большой долей вероятности конвесии в control и personalization не равны и  имеют статистически значимые отличия